In [ ]:
%pip install torch
%pip install fairscale
%pip install fire
%pip install sentencepiece

In [1]:
import subprocess
import json
import os
from dotenv import load_dotenv

load_dotenv()

LLM_DATA_FEED = os.environ['LLM_DATA_FEED']
LLM_DATA_REPLY = os.environ['LLM_DATA_REPLY']

LLAMA_RUNNER = os.environ['LLAMA_RUNNER']
GPT_RUNNER = os.environ['GPT_RUNNER']


'''
LLAMA PIPELINE
'''

def llama(input):

    with open(LLM_DATA_FEED, 'w') as file:
        json.dump(input, file, indent=4)

    arguments = ['torchrun', 
                '--nproc_per_node=1', 
                f'{LLAMA_RUNNER}', 
                '--max_seq_len=512', 
                '--max_batch_size=6']
    result = subprocess.run(arguments, capture_output=True, text=True)

    print("Errors:", result.stderr)
    print("Return Code:", result.returncode)

    return result.stdout

def parse_llama_result(data:str, begin_key=None, end_key=None, inclusive_begin=True):
    story = data[data.find('[@RESPONSEBEGIN]'):data.find('[@RESPONSEEND]')]

    # trims to between begin key and end key(if they exist)
    begin_index = 0
    if begin_key is not None:
        begin_index = story.find(begin_key)
        if not inclusive_begin:
            begin_index += len(begin_key)

    end_index = len(story)
    if end_key is not None:
        end_index = story.find(end_key)

    story = story[begin_index:end_index]

    return story

def parse_characters(data:str):
    characters = []
    last_begin = 0
    while(True):
        begin = data.find('<', last_begin)
        print(f'begin: {begin}')
        if begin == -1:
            break
        last_begin = begin + 3

        end = data.find('>', begin)
        characters.append(data[begin + 1:end])
    print(f"found characters: {characters}")
    return characters


'''
GPT PIPELINE
'''

def gpt(input):
    with open(LLM_DATA_FEED, 'w') as file:
        json.dump(input, file, indent=4)

    arguments = ["python3", f"{GPT_RUNNER}"]

    result = subprocess.run(arguments, capture_output=True, text=True)

    print("Errors:", result.stderr)
    print("Return Code:", result.returncode)

    with open(LLM_DATA_REPLY, 'r') as file:
        data = json.load(file)
    return data

    

In [2]:
def llama_pipeline(prompt):

    # things needed:
    #   characters list
    #   series of captions
    #   series of dialogue and the character that says it

    # obtaining story:
    data = [
        [
            {"role": "user", "content": f"write a story about {prompt}, breaking apart each scene into its own paragraph and label it with [SCENE <number>]. End the past paragraph with [END]"}
        ],
    ]
    story = llama(data)
    story_parsed = parse_llama_result(story, "[SCENE", "[END]")

    # obtaining characters
    data = [
        [
            {"role": "user", "content": f"here is a story: {story_parsed}\nGenerate a list of characters that may be in this story by providing their names between triangle brackets as such in the following format: <name>"}
        ],
    ]
    characters = llama(data)
    characters = parse_characters(characters)

    # generating captions
    data = [
        [
            {"role": "user", "content": f"here is a story: {story_parsed}\nGenerate a caption for each scene, labelling each caption with [SCENE <number>]. End the last caption with [END]"}
        ],
    ]
    captions = llama(data)
    captions_parsed = parse_llama_result(captions, "[SCENE", "[END]")

    # generating dialogue
    data = [
        [
            {"role": "user", "content": f"here is a story: {story_parsed}\nGenerate a set of dialogues ranging from 1 to 3 for each scene, labelling each dialogue with [SCENE <number>][CHARACTER <character name>]. Use characters from the following pool: {','.join(characters)}"}
        ],
    ]
    dialogue = llama(data)
    dialogue_parsed = parse_llama_result(dialogue, "[SCENE", "[END]")

    # TODO: fix the parsers

    print(f"story_parsed: \n{story_parsed}\nEND")
    print(f"characters parsed: \n{characters}\nEND")
    print(f"captions_parsed: \n{captions_parsed}\nEND")
    print(f"dialogue_parsed: \n{dialogue_parsed}\nEND")

    # TODO: combine_results and return


def gpt_pipeline(prompt):
    example = """
{
    "characters": [
        {"name": "Radke"}
    ],
    "script": [ 
        {"caption": "A sheep looking at cheese in a supermarket.", 
        "dialogue": [
            {"character": "Radke", "text": "In the mist-enshrouded hills of an ancient land, there lies a mystery as old as time itself. Behold the enigmatic sheep, creatures shrouded in the lore and legend of yesteryears."}
        ]
        }
    ]
}"""

    data = [
        {"role": "system", "content": "You are a TV show writer."},
        {"role": "user", "content": f"generate a story about {prompt} by filling in a json file, remember that captions should be physical descriptions of an image, and the script should contain around 20 captions. The following json example shows all the fields you should create and fill in: {example}"}
    ]
    response = gpt(data)
    return response

In [3]:

prompt = 'animals overrunning a village and destroying its crops'
story = gpt_pipeline(prompt)
print(f'response:\n{story}')

Errors: 
Return Code: 0
response:
{'characters': [{'name': 'FarmerJohn'}, {'name': 'OldManSam'}], 'script': [{'caption': 'Sunrise breaking over a quaint rural village with blooming crops.', 'dialogue': [{'character': 'FarmerJohn', 'text': 'Nothing’s more fulfilling than harvesting what you sow, eh Sam?'}]}, {'caption': 'A flock of sheep infiltrating a vegetable garden and nibbling away.', 'dialogue': [{'character': 'OldManSam', 'text': 'Those damn sheep again! They’re chewing through our livelihood!'}]}, {'caption': 'Chickens pecking away at a perfectly baked pie left to cool off.', 'dialogue': [{'character': 'FarmerJohn', 'text': "And the chickens have developed quite a sweet tooth. They're threatening our pies now."}]}, {'caption': 'Pigs on a rampage, digging up tuberous crops.', 'dialogue': [{'character': 'OldManSam', 'text': "The pigs aren't any better, digging up all our taters."}]}, {'caption': 'A large herd of cattle trampling through the corn field.', 'dialogue': [{'character':